# **Setup**

## **Install and Launch The Postgres Server**

In [1]:
!apt -qq install postgresql
!service postgresql start


The following additional packages will be installed:
  libcommon-sense-perl libjson-perl libjson-xs-perl libtypes-serialiser-perl logrotate netbase
  postgresql-14 postgresql-client-14 postgresql-client-common postgresql-common ssl-cert sysstat
Suggested packages:
  bsd-mailx | mailx postgresql-doc postgresql-doc-14 isag
The following NEW packages will be installed:
  libcommon-sense-perl libjson-perl libjson-xs-perl libtypes-serialiser-perl logrotate netbase
  postgresql postgresql-14 postgresql-client-14 postgresql-client-common postgresql-common ssl-cert
  sysstat
0 upgraded, 13 newly installed, 0 to remove and 10 not upgraded.
Need to get 18.3 MB of archives.
After this operation, 51.5 MB of additional disk space will be used.
Preconfiguring packages ...
Selecting previously unselected package logrotate.
(Reading database ... 120880 files and directories currently installed.)
Preparing to unpack .../00-logrotate_3.19.0-1ubuntu1.1_amd64.deb ...
Unpacking logrotate (3.19.0-1ubuntu1.1

## **Create Postgres User and Database**

In [2]:
!sudo -u postgres psql -c "CREATE USER eva WITH SUPERUSER PASSWORD 'password'"
!sudo -u postgres psql -c "CREATE DATABASE evadb"

CREATE ROLE
CREATE DATABASE


## **Installing EvaDB and Necessary Packages / Dependencies**

In [3]:
%pip install fastapi
%pip install kaleido
%pip install python-multipart
%pip install jedi
%pip install uvicorn
%pip install --upgrade pip setuptools wheel
%pip install --upgrade evadb

%pip install psycopg2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.8.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 9.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that ar

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.7/578.7 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.6/111.6 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 89.1 MB/s eta 0:00:00


## **Setting Up EvaDB**

In [4]:
# CS 4420 EvaDB Project 2
# Andrew Titus

# Import the EvaDB package
import evadb
#from evadb import ludwig
import numpy as np
import psycopg2

# Connect to EvaDB and get a database cursor for running queries
cursor = evadb.connect().cursor()

# List all the built-in functions in EvaDB
print(cursor.query("SHOW FUNCTIONS;").df())

Downloading: "http://ml.cs.tsinghua.edu.cn/~chenxi/pytorch-models/mnist-b07bb66b.pth" to /root/.cache/torch/hub/checkpoints/mnist-b07bb66b.pth
100%|██████████| 1.03M/1.03M [00:01<00:00, 1.01MB/s]
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


                     name                                             inputs  \
0    MnistImageClassifier                 [data NDARRAY ANYTYPE (3, 28, 28)]   
1  FastRCNNObjectDetector        [Frame_Array NDARRAY UINT8 (3, None, None)]   
2              ArrayCount   [Input_Array NDARRAY ANYTYPE (), Search_Key ANY]   
3                    Crop  [Frame_Array NDARRAY UINT8 (3, None, None), bb...   
4                    Open                                    [img_path TEXT]   
5                 ChatGPT  [query NDARRAY STR (1,), content NDARRAY STR (...   
6         StableDiffusion                       [prompt NDARRAY STR (None,)]   
7                   DallE                       [prompt NDARRAY STR (None,)]   

                                             outputs             type  \
0                                       [label TEXT]   Classification   
1  [labels NDARRAY STR (None,), bboxes NDARRAY FL...   Classification   
2                                [key_count INTEGER]  Ndarra

## **Setting Up PostgreSQL Data Source with EvaDB**

In [5]:
# Connects PostgreSQL LoanDefault Database to EvaDB
# postgres at port 5432 localhost -> ngrok server at tcp://4.tcp.ngrok.io:17588
params = {
    "user": "eva",
    "password": "password", # enter PostgreSQL password here
    "host": "localhost",
    "port": "5432",
    "database": "evadb",
}
query = f"CREATE DATABASE IF NOT EXISTS pg WITH ENGINE = 'postgres', PARAMETERS = {params};"
#print(query)
print(cursor.query(query).df())

                                                0
0  The database pg has been successfully created.


## **Download CSV from Kaggle Dataset**

Note: You may have to manually upload the CSV into the content folder. The first two lines were originally uncommented.

In [6]:
!mkdir -p content
!wget --header="Authorization: ApiKey 0ad0f3f3a77ada5f751f0bded1e68105" -nc -O /content/Loan_Default.csv https://www.kaggle.com/datasets/yasserh/loan-default-dataset/download/Loan_Default.csv

#!wget -nc -O /content/Loan_Default.csv https://www.kaggle.com/datasets/yasserh/loan-default-dataset/download/Loan_Default.csv



--2023-11-25 19:59:39--  https://www.kaggle.com/datasets/yasserh/loan-default-dataset/download/Loan_Default.csv
Resolving www.kaggle.com (www.kaggle.com)... 35.244.233.98
Connecting to www.kaggle.com (www.kaggle.com)|35.244.233.98|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /account/login?titleType=dataset-downloads&showDatasetDownloadSkip=False&messageId=datasetsWelcome&returnUrl=%2Fdatasets%2Fyasserh%2Floan-default-dataset%3Fresource%3Ddownload [following]
--2023-11-25 19:59:39--  https://www.kaggle.com/account/login?titleType=dataset-downloads&showDatasetDownloadSkip=False&messageId=datasetsWelcome&returnUrl=%2Fdatasets%2Fyasserh%2Floan-default-dataset%3Fresource%3Ddownload
Reusing existing connection to www.kaggle.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘/content/Loan_Default.csv’

/content/Loan_Defau     [ <=>                ]   4.80K  --.-KB/s    in 0.008s  

2023-11-25 19:59:39 (610 KB/

# **Data Collection**

The following queries set up the table of data from the imported Kaggle dataset. An example query displays the first 15 rows of the table.

In [7]:
cursor.query("""
  USE pg {
    CREATE TABLE IF NOT EXISTS loan_default (id VARCHAR(20), year VARCHAR(20), loan_limit VARCHAR(20),
    gender VARCHAR(20), approv_in_adv VARCHAR(10), loan_type VARCHAR(10), loan_purpose VARCHAR(10),
    credit_worthiness VARCHAR(10), open_credit VARCHAR(10), business_or_commercial VARCHAR(10),
    loan_amount FLOAT, rate_of_interest FLOAT, interest_rate_spread FLOAT, upfront_charges FLOAT, term FLOAT,
    neg_ammortization VARCHAR(10), interest_only VARCHAR(10), lump_sum_payment VARCHAR(10),
    property_value FLOAT, construction_type VARCHAR(10), occupancy_type VARCHAR(10), secured_by VARCHAR(10),
    total_units VARCHAR(10), income FLOAT, credit_type VARCHAR(10), credit_score INT,
    co_applicant_credit_type VARCHAR(10), age VARCHAR(10), submission_of_application VARCHAR(10), ltv FLOAT,
    region VARCHAR(10), security_type VARCHAR(15), status INTEGER, dtir1 FLOAT)
  }
""").df()


,status
0,success


In [24]:
cursor.query("""
  USE pg {
    COPY loan_default (id, year, loan_limit,
    gender, approv_in_adv, loan_type, loan_purpose,
    credit_worthiness, open_credit, business_or_commercial,
    loan_amount, rate_of_interest, interest_rate_spread, upfront_charges, term,
    neg_ammortization, interest_only, lump_sum_payment,
    property_value, construction_type, occupancy_type, secured_by,
    total_units, income, credit_type, credit_score,
    co_applicant_credit_type, age, submission_of_application, ltv,
    region, security_type, status, dtir1)
    FROM '/content/Loan_Default.csv'
    DELIMITER ',' CSV HEADER
  }
""").df()

,status
0,success


In [25]:
cursor.query("SELECT * FROM pg.loan_default LIMIT 15;").df()

/usr/local/lib/python3.10/dist-packages/evadb/third_party/databases/postgres/postgres_handler.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tables_df = pd.read_sql_query(query, self.connection)
/usr/local/lib/python3.10/dist-packages/evadb/third_party/databases/postgres/postgres_handler.py:111: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  columns_df = pd.read_sql_query(query, self.connection)
/usr/local/lib/python3.10/dist-packages/evadb/third_party/databases/postgres/postgres_handler.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please conside

,income,interest_rate_spread,credit_score,property_value,upfront_charges,term,ltv,loan_amount,rate_of_interest,status,...,year,loan_limit,gender,approv_in_adv,loan_type,loan_purpose,credit_worthiness,open_credit,business_or_commercial,neg_ammortization
0,3300.0,NaN,567,NaN,NaN,360.0,NaN,296500.0,NaN,1,...,2019,cf,Male,nopre,type1,p4,l1,nopc,nob/c,not_neg
1,11160.0,NaN,635,NaN,NaN,180.0,NaN,266500.0,NaN,1,...,2019,cf,Male,pre,type2,p3,l1,nopc,b/c,not_neg
2,3780.0,NaN,824,168000.0,NaN,180.0,81.250000,136500.0,NaN,1,...,2019,cf,Male,nopre,type2,p3,l1,nopc,b/c,not_neg
3,11640.0,NaN,638,NaN,NaN,360.0,NaN,696500.0,NaN,1,...,2019,cf,Male,nopre,type1,p4,l1,nopc,nob/c,not_neg
4,8040.0,NaN,660,NaN,NaN,360.0,NaN,306500.0,NaN,1,...,2019,cf,Male,nopre,type1,p1,l1,nopc,nob/c,neg_amm
5,NaN,NaN,732,758000.0,NaN,360.0,86.609499,656500.0,NaN,1,...,2019,cf,Male,pre,type2,p4,l1,nopc,b/c,not_neg
6,6660.0,NaN,544,NaN,NaN,360.0,NaN,106500.0,NaN,1,...,2019,cf,Joint,nopre,type1,p4,l1,nopc,nob/c,not_neg
7,3300.0,NaN,762,NaN,NaN,360.0,NaN,156500.0,NaN,1,...,2019,cf,Sex Not Available,pre,type1,p2,l1,nopc,nob/c,not_neg
8,3000.0,NaN,697,NaN,NaN,300.0,NaN,186500.0,NaN,1,...,2019,cf,Male,nopre,type1,p1,l2,nopc,nob/c,neg_amm
9,NaN,NaN,687,308000.0,NaN,360.0,102.759740,316500.0,NaN,1,...,2019,cf,Male,nopre,type3,p4,l1,nopc,nob/c,not_neg


# **ML Model**

First, we need to install a few extra packages from evadb, including ludwig.

In [26]:
%pip install --quiet "evadb[document, forecasting, ludwig]"

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.2/682.2 kB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.1/98.1 

## **Create Training Function**

In [28]:
cursor.query("""CREATE OR REPLACE FUNCTION DefaultPredictor FROM
( SELECT * FROM pg.loan_default)
TYPE Ludwig
PREDICT 'status'
TIME_LIMIT 3600;
""").df()

/usr/local/lib/python3.10/dist-packages/evadb/third_party/databases/postgres/postgres_handler.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tables_df = pd.read_sql_query(query, self.connection)
/usr/local/lib/python3.10/dist-packages/evadb/third_party/databases/postgres/postgres_handler.py:111: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  columns_df = pd.read_sql_query(query, self.connection)
/usr/local/lib/python3.10/dist-packages/evadb/third_party/databases/postgres/postgres_handler.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please conside

/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


Analyzing fields: 100%|██████████| 34/34 [00:04<00:00,  7.12it/s]
/usr/local/lib/python3.10/dist-packages/ludwig/schema/model_types/utils.py:242: UserWarning: Can't utilize `early_stop` while using a hyperopt scheduler. Setting early stop to -1.
  warnings.warn("Can't utilize `early_stop` while using a hyperopt scheduler. Setting early stop to -1.")
2023-11-25 20:11:33,144	INFO worker.py:1616 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
/usr/local/lib/python3.10/dist-packages/ludwig/schema/model_types/utils.py:242: UserWarning: Can't utilize `early_stop` while using a hyperopt scheduler. Setting early stop to -1.
  warnings.warn("Can't utilize `early_stop` while using a hyperopt scheduler. Setting early stop to -1.")
/usr/local/lib/python3.10/dist-packages/ludwig/hyperopt/utils.py:204: RuntimeWarning: All hyperopt parameters in Ludwig config are using grid_search space, but number of samples (10) is greater than 1. This will result in 9 duplicate trials being 

(_register pid=6224) /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


(_register pid=6224) /usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
(_register pid=6224)   warn("The installed version of bitsandbytes was compiled without GPU support. "
(_register pid=6224) 2023-11-25 20:12:24.325546: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(_register pid=6224) 2023-11-25 20:12:24.325604: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(_register pid=6224) 2023-11-25 20:12:24.325634: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register fact

...

(pid=15719) /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


(pid=15719) /usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
(pid=15719)   warn("The installed version of bitsandbytes was compiled without GPU support. "
(pid=15719) 2023-11-25 20:47:37.501962: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=15719) 2023-11-25 20:47:37.502042: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=15719) 2023-11-25 20:47:37.502092: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already be

(pid=16767) /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


(pid=16767) 2023-11-25 20:51:41.123030: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=16767) 2023-11-25 20:51:41.123107: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=16767) 2023-11-25 20:51:41.123152: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(pid=16767) 2023-11-25 20:51:43.888785: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
(run_experiment_trial pid=16767) /usr/local/lib/python3.10/dist-packages/ludwig/schema/model_types/utils.py:242: UserWarning: Can't utilize `early_stop` while using a hyperopt scheduler. Set

,0
0,Function DefaultPredictor added to the database.
1,Validation Score: 1.0
2,Training time: 3943 secs.


## **Manually Entering Data**
This block enters data manually and adds tuples to the prediction table. If you want to upload multiple tuples to the prediction table, comment out the line to clear predictions from the table.

In [29]:
cursor.query("""
  USE pg {
    CREATE TABLE IF NOT EXISTS home_loan_default_predictions (id VARCHAR(20), year VARCHAR(20), loan_limit VARCHAR(20),
    gender VARCHAR(20), approv_in_adv VARCHAR(10), loan_type VARCHAR(10), loan_purpose VARCHAR(10),
    credit_worthiness VARCHAR(10), open_credit VARCHAR(10), business_or_commercial VARCHAR(10),
    loan_amount FLOAT, rate_of_interest FLOAT, interest_rate_spread FLOAT, upfront_charges FLOAT, term FLOAT,
    neg_ammortization VARCHAR(10), interest_only VARCHAR(10), lump_sum_payment VARCHAR(10),
    property_value FLOAT, construction_type VARCHAR(10), occupancy_type VARCHAR(10), secured_by VARCHAR(10),
    total_units VARCHAR(10), income FLOAT, credit_type VARCHAR(10), credit_score INT,
    co_applicant_credit_type VARCHAR(10), age VARCHAR(10), submission_of_application VARCHAR(10), ltv FLOAT,
    region VARCHAR(10), security_type VARCHAR(15), status INTEGER, dtir1 FLOAT)
  }
""").df()

# Clears predictions table if not empty
#cursor.query("USE pg {DELETE FROM home_loan_default_predictions};").df()

,status
0,success


In [43]:
# Clears predictions table if not empty
cursor.query("USE pg {DELETE FROM home_loan_default_predictions};").df()

cursor.query("""
  USE pg {
    INSERT INTO home_loan_default_predictions
    (id, year, loan_limit, gender, approv_in_adv, loan_type, loan_purpose,
     credit_worthiness, open_credit, business_or_commercial, loan_amount,
     rate_of_interest, interest_rate_spread, upfront_charges, term,
     neg_ammortization, interest_only, lump_sum_payment, property_value,
     construction_type, occupancy_type, secured_by, total_units, income,
     credit_type, credit_score, co_applicant_credit_type, age,
     submission_of_application, ltv, region, security_type, status, dtir1)
    VALUES (
      '173660', '2019', 'cf', 'Male', 'nopre', 'type2', 'P1', 'l1', 'nopc', 'b/c',
      '206500', NULL, NULL, NULL, '360', 'not_neg', 'not_int', 'lpsm', NULL, 'sb', 'pr',
      'home', '1U', '4980', 'EQUI', '552', 'EXP', '55-64', 'to_inst', NULL, 'North', 'direct', 0, NULL
    )
  }
""").df()

cursor.query("""
  USE pg {
    INSERT INTO home_loan_default_predictions
    (id, year, loan_limit, gender, approv_in_adv, loan_type, loan_purpose,
     credit_worthiness, open_credit, business_or_commercial, loan_amount,
     rate_of_interest, interest_rate_spread, upfront_charges, term,
     neg_ammortization, interest_only, lump_sum_payment, property_value,
     construction_type, occupancy_type, secured_by, total_units, income,
     credit_type, credit_score, co_applicant_credit_type, age,
     submission_of_application, ltv, region, security_type, status, dtir1)
    VALUES (
      '173661', '2019', 'cf', 'MALE', 'nopre', 'type1', 'P4', 'l1', 'nopc', 'nob/c',
      '526500', '3.99', '0.3849', '635.14', 360, 'not_neg', 'not_int', 'not_lpsm', '658000', 'sb', 'pr',
      'home', '1U', '11400', 'CRIF', '579', 'CIB', '<25', 'not_inst', '80.0152', 'south', 'direct', 0, '29'
    )
  }
""").df()

cursor.query("""
  USE pg {
    INSERT INTO home_loan_default_predictions
    (id, year, loan_limit, gender, approv_in_adv, loan_type, loan_purpose,
     credit_worthiness, open_credit, business_or_commercial, loan_amount,
     rate_of_interest, interest_rate_spread, upfront_charges, term,
     neg_ammortization, interest_only, lump_sum_payment, property_value,
     construction_type, occupancy_type, secured_by, total_units, income,
     credit_type, credit_score, co_applicant_credit_type, age,
     submission_of_application, ltv, region, security_type, status, dtir1)
    VALUES (
      '173661', '2019', 'cf', 'MALE', 'nopre', 'type1', 'P4', 'l1', 'nopc', 'nob/c',
      '526500', '3.99', '0.3849', '635.14', 360, 'not_neg', 'not_int', 'lpsm', '658000', 'sb', 'pr',
      'home', '1U', '11400', 'CRIF', '579', 'CIB', '<25', 'not_inst', '80.0152', 'south', 'direct', 0, '29'
    )
  }
""").df()

cursor.query("""
  USE pg {
    INSERT INTO home_loan_default_predictions
    (id, year, loan_limit, gender, approv_in_adv, loan_type, loan_purpose,
     credit_worthiness, open_credit, business_or_commercial, loan_amount,
     rate_of_interest, interest_rate_spread, upfront_charges, term,
     neg_ammortization, interest_only, lump_sum_payment, property_value,
     construction_type, occupancy_type, secured_by, total_units, income,
     credit_type, credit_score, co_applicant_credit_type, age,
     submission_of_application, ltv, region, security_type, status, dtir1)
    VALUES (
      '173661', '2019', 'cf', 'FEMALE', 'nopre', 'type1', 'P4', 'l1', 'nopc', 'nob/c',
      '526500', '3.99', '0.3849', '635.14', 360, 'not_neg', 'not_int', 'not_lpsm', '658000', 'sb', 'pr',
      'home', '1U', '11400', 'CRIF', '579', 'CIB', '<25', 'not_inst', '80.0152', 'south', 'direct', 0, '29'
    )
  }
""").df()

cursor.query("""SELECT * FROM pg.home_loan_default_predictions;""").df()

/usr/local/lib/python3.10/dist-packages/evadb/third_party/databases/postgres/postgres_handler.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tables_df = pd.read_sql_query(query, self.connection)
/usr/local/lib/python3.10/dist-packages/evadb/third_party/databases/postgres/postgres_handler.py:111: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  columns_df = pd.read_sql_query(query, self.connection)
/usr/local/lib/python3.10/dist-packages/evadb/third_party/databases/postgres/postgres_handler.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please conside

,income,interest_rate_spread,credit_score,property_value,upfront_charges,term,ltv,loan_amount,rate_of_interest,status,...,year,loan_limit,gender,approv_in_adv,loan_type,loan_purpose,credit_worthiness,open_credit,business_or_commercial,neg_ammortization
0,4980.0,NaN,552,NaN,NaN,360.0,NaN,206500.0,NaN,0,...,2019,cf,Male,nopre,type2,P1,l1,nopc,b/c,not_neg
1,11400.0,0.3849,579,658000.0,635.14,360.0,80.0152,526500.0,3.99,0,...,2019,cf,MALE,nopre,type1,P4,l1,nopc,nob/c,not_neg
2,11400.0,0.3849,579,658000.0,635.14,360.0,80.0152,526500.0,3.99,0,...,2019,cf,MALE,nopre,type1,P4,l1,nopc,nob/c,not_neg
3,11400.0,0.3849,579,658000.0,635.14,360.0,80.0152,526500.0,3.99,0,...,2019,cf,FEMALE,nopre,type1,P4,l1,nopc,nob/c,not_neg


In [44]:
cursor.query("""SELECT DefaultPredictor(*) FROM pg.home_loan_default_predictions;""").df()

/usr/local/lib/python3.10/dist-packages/evadb/third_party/databases/postgres/postgres_handler.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tables_df = pd.read_sql_query(query, self.connection)
/usr/local/lib/python3.10/dist-packages/evadb/third_party/databases/postgres/postgres_handler.py:111: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  columns_df = pd.read_sql_query(query, self.connection)
/usr/local/lib/python3.10/dist-packages/evadb/third_party/databases/postgres/postgres_handler.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please conside

(dask:('map-da61c0ef7d5f6a873ba48755315d1a76', 0) pid=6224) 
(dask:('map-da61c0ef7d5f6a873ba48755315d1a76', 0) pid=6224)                 Encountered unknown symbol 'MALE' for 'gender' during category
(dask:('map-da61c0ef7d5f6a873ba48755315d1a76', 0) pid=6224)                 feature preprocessing. This should never happen during training. If this happens during
(dask:('map-da61c0ef7d5f6a873ba48755315d1a76', 0) pid=6224)                 inference, this may be an indication that not all possible symbols were present in your
(dask:('map-da61c0ef7d5f6a873ba48755315d1a76', 0) pid=6224)                 training set. Consider re-splitting your data to ensure full representation, or setting
(dask:('map-da61c0ef7d5f6a873ba48755315d1a76', 0) pid=6224)                 preprocessing.most_common parameter to be smaller than this feature's total vocabulary
(dask:('map-da61c0ef7d5f6a873ba48755315d1a76', 0) pid=6224)                 size, 4, which will ensure that the model is architected and
(dask:('

(dask:('len-chunk-3813eb059ef7da42bb4dab028b23e5d4-aaffcb948e84ff3b4318766d851937d1', 0) pid=35922)   warn("The installed version of bitsandbytes was compiled without GPU support. "
(dask:('len-chunk-3813eb059ef7da42bb4dab028b23e5d4-aaffcb948e84ff3b4318766d851937d1', 0) pid=35922)   warn("The installed version of bitsandbytes was compiled without GPU support. "


(dask:('len-chunk-3813eb059ef7da42bb4dab028b23e5d4-aaffcb948e84ff3b4318766d851937d1', 0) pid=35922) /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
(dask:('map-da61c0ef7d5f6a873ba48755315d1a76', 0) pid=6224) 
(dask:('map-da61c0ef7d5f6a873ba48755315d1a76', 0) pid=6224)                 Encountered unknown symbol 'MALE' for 'gender' during category
(dask:('map-da61c0ef7d5f6a873ba48755315d1a76', 0) pid=6224)                 feature preprocessing. This should never happen during training. If this happens during
(dask:('map-da61c0ef7d5f6a873ba48755315d1a76', 0) pid=6224)                 inference, this may be an indication that not all possible symbols were present in your
(dask:('map-da61c0ef7d5f6a873ba48755315d1a76', 0) pid=6224)                 training set. Consider re-splitting your data to ensure full representation, or setting
(dask:('map-da61c0ef7d5f6a873ba48755315d1a76', 0) pid=6224)                 preprocessing.mo

2023-11-25 22:04:01,659	INFO streaming_executor.py:83 -- Executing DAG InputDataBuffer[Input] -> ActorPoolMapOperator[MapBatches(BatchInferModel)]
2023-11-25 22:04:01,666	INFO streaming_executor.py:84 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)
2023-11-25 22:04:01,725	INFO actor_pool_map_operator.py:99 -- MapBatches(BatchInferModel): Waiting for 1 pool actors to start...


...

(MapWorker(MapBatches(BatchInferModel)) pid=36069) /usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
(MapWorker(MapBatches(BatchInferModel)) pid=36069)   warn("The installed version of bitsandbytes was compiled without GPU support. "


(MapWorker(MapBatches(BatchInferModel)) pid=36069) /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


2023-11-25 22:04:20,721	INFO streaming_executor.py:83 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[MapBatches(postprocess_batch)]
2023-11-25 22:04:20,723	INFO streaming_executor.py:84 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)


Running 0:   0%|          | 0/1 [00:00<?, ?it/s]

,status_predictions
0,True
1,False
2,False
3,False


In [46]:
cursor.query("""
  SELECT status, status_predictions FROM pg.home_loan_default_predictions
  JOIN LATERAL DefaultPredictor(*) AS Predicted(status_predictions) LIMIT 15;
""").df()

/usr/local/lib/python3.10/dist-packages/evadb/third_party/databases/postgres/postgres_handler.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tables_df = pd.read_sql_query(query, self.connection)
/usr/local/lib/python3.10/dist-packages/evadb/third_party/databases/postgres/postgres_handler.py:111: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  columns_df = pd.read_sql_query(query, self.connection)
/usr/local/lib/python3.10/dist-packages/evadb/third_party/databases/postgres/postgres_handler.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please conside

(dask:('map-1292fea419e42f57497cf34781ae885c', 0) pid=38057) 
(dask:('map-1292fea419e42f57497cf34781ae885c', 0) pid=38057)                 Encountered unknown symbol 'MALE' for 'gender' during category
(dask:('map-1292fea419e42f57497cf34781ae885c', 0) pid=38057)                 feature preprocessing. This should never happen during training. If this happens during
(dask:('map-1292fea419e42f57497cf34781ae885c', 0) pid=38057)                 inference, this may be an indication that not all possible symbols were present in your
(dask:('map-1292fea419e42f57497cf34781ae885c', 0) pid=38057)                 training set. Consider re-splitting your data to ensure full representation, or setting
(dask:('map-1292fea419e42f57497cf34781ae885c', 0) pid=38057)                 preprocessing.most_common parameter to be smaller than this feature's total vocabulary
(dask:('map-1292fea419e42f57497cf34781ae885c', 0) pid=38057)                 size, 4, which will ensure that the model is architected and
(

2023-11-25 22:13:02,331	INFO streaming_executor.py:83 -- Executing DAG InputDataBuffer[Input] -> ActorPoolMapOperator[MapBatches(BatchInferModel)]
2023-11-25 22:13:02,337	INFO streaming_executor.py:84 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)
2023-11-25 22:13:02,402	INFO actor_pool_map_operator.py:99 -- MapBatches(BatchInferModel): Waiting for 1 pool actors to start...


Running 0:   0%|          | 0/1 [00:00<?, ?it/s]

(MapWorker(MapBatches(BatchInferModel)) pid=38527) /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
(dask:('map-1292fea419e42f57497cf34781ae885c', 0) pid=6224)                  [repeated 6x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(dask:('map-1292fea419e42f57497cf34781ae885c', 0) pid=6224)                 Encountered unknown symbol 'MALE' for 'gender' during category [repeated 2x across cluster]
(dask:('map-1292fea419e42f57497cf34781ae885c', 0) pid=6224)                 feature preprocessing. This should never happen during training. If this happens during [repeated 3x across cluster]
(dask:('map-1292fea419e42f57497cf34781ae885c', 0) pid=6224)                 inference, this may be an indication that not all possible symbols were present in your [repeat

(MapWorker(MapBatches(BatchInferModel)) pid=38527)   warn("The installed version of bitsandbytes was compiled without GPU support. "
(MapWorker(MapBatches(BatchInferModel)) pid=38527)   warn("The installed version of bitsandbytes was compiled without GPU support. "
2023-11-25 22:13:26,517	INFO streaming_executor.py:83 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[MapBatches(postprocess_batch)]
2023-11-25 22:13:26,524	INFO streaming_executor.py:84 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)


Running 0:   0%|          | 0/1 [00:00<?, ?it/s]

,status,status_predictions
0,0,True
1,0,False
2,0,False
3,0,False


## **Predicting The First 15 Columns Of The Table**

In this attempt to use the Ludwig model, we will predict the status for the first 15 columns of the table and then compare the results with the actual status from the table.



In [47]:
cursor.query("SELECT DefaultPredictor(*) FROM pg.loan_default LIMIT 15;").df()

/usr/local/lib/python3.10/dist-packages/evadb/third_party/databases/postgres/postgres_handler.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tables_df = pd.read_sql_query(query, self.connection)
/usr/local/lib/python3.10/dist-packages/evadb/third_party/databases/postgres/postgres_handler.py:111: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  columns_df = pd.read_sql_query(query, self.connection)
/usr/local/lib/python3.10/dist-packages/evadb/third_party/databases/postgres/postgres_handler.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please conside

Running 0:   0%|          | 0/1 [00:00<?, ?it/s]

(MapWorker(MapBatches(BatchInferModel)) pid=40511) /usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
(MapWorker(MapBatches(BatchInferModel)) pid=40511)   warn("The installed version of bitsandbytes was compiled without GPU support. "


(MapWorker(MapBatches(BatchInferModel)) pid=40511) /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


2023-11-25 22:21:02,538	INFO streaming_executor.py:83 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[MapBatches(postprocess_batch)]
2023-11-25 22:21:02,541	INFO streaming_executor.py:84 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)


Running 0:   0%|          | 0/1 [00:00<?, ?it/s]

,status_predictions
0,True
1,True
2,True
3,True
4,True
5,True
6,True
7,True
8,True
9,True


In [ ]:
cursor.query("""
  SELECT status, status_predictions FROM pg.loan_default
  JOIN LATERAL DefaultPredictor(*) AS Predicted(status_predictions) LIMIT 15;
""").df()

/usr/local/lib/python3.10/dist-packages/evadb/third_party/databases/postgres/postgres_handler.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tables_df = pd.read_sql_query(query, self.connection)
/usr/local/lib/python3.10/dist-packages/evadb/third_party/databases/postgres/postgres_handler.py:111: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  columns_df = pd.read_sql_query(query, self.connection)
/usr/local/lib/python3.10/dist-packages/evadb/third_party/databases/postgres/postgres_handler.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please conside

Running 0:   0%|          | 0/1 [00:00<?, ?it/s]

(MapWorker(MapBatches(BatchInferModel)) pid=24926) /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


(MapWorker(MapBatches(BatchInferModel)) pid=24926)   warn("The installed version of bitsandbytes was compiled without GPU support. "
(MapWorker(MapBatches(BatchInferModel)) pid=24926)   warn("The installed version of bitsandbytes was compiled without GPU support. "
2023-11-25 06:53:28,758	INFO streaming_executor.py:83 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[MapBatches(postprocess_batch)]
2023-11-25 06:53:28,761	INFO streaming_executor.py:84 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)


Running 0:   0%|          | 0/1 [00:00<?, ?it/s]

,status,status_predictions
0,1,True
1,1,True
2,0,False
3,0,False
4,0,False
5,0,False
6,0,False
7,0,False
8,1,True
9,1,True
